# Imputación con mediana

## Preparar librerias

In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import random
import warnings
#import math
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import Normalizer
import importlib.util

# Especifica la ruta al archivo del módulo
ruta_al_modulo = './util/auxi.py'

# Importa el módulo
spec = importlib.util.spec_from_file_location("auxi", ruta_al_modulo)
auxi = importlib.util.module_from_spec(spec)
spec.loader.exec_module(auxi)

## Leer datos originales

Hay que leer el dataset original para comparar los datos imputados con los reales

Temperatura

In [2]:
warnings.filterwarnings('ignore', category=SyntaxWarning)  # Porque el \d de \data y \daily lo toma como un valor especial
dataframe_temperatura = pd.read_csv("..\data\daily-minimum-temperatures-in-me.csv", header=0, index_col=0, parse_dates=True)
serie_temperatura = dataframe_temperatura.squeeze('columns')
serie_temperatura = serie_temperatura.resample('D').mean()
serie_temperatura['1984-12-31'] = (serie_temperatura['1984-12-30'] + serie_temperatura['1985-01-01'])/2
serie_temperatura['1988-12-31'] = (serie_temperatura['1988-12-30'] + serie_temperatura['1989-01-01'])/2
serie_temperatura.head()

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\alaba\AppData\Local\Temp\ipykernel_5488\790941083.py:2: SyntaxWarning: invalid escape sequence '\d'
  dataframe_temperatura = pd.read_csv("..\data\daily-minimum-temperatures-in-me.csv", header=0, index_col=0, parse_dates=True)


Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
Freq: D, Name: Daily minimum temperatures in Melbourne, Australia, 1981-1990, dtype: float64

Cardiovascular

In [3]:
dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)
serie_cardiovascular = dataframe_cardiovascular.squeeze('columns')
serie_cardiovascular = serie_cardiovascular.astype(float)
serie_cardiovascular.head()

C:\Users\alaba\AppData\Local\Temp\ipykernel_5488\3747356855.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)


Date
2017-08-03 00:00:00    49136.0
2017-08-03 01:00:00    46450.0
2017-08-03 02:00:00    46355.0
2017-08-03 03:00:00    43748.0
2017-08-03 04:00:00    42281.0
Name: ads, dtype: float64

Manchas solares

In [4]:
dataframe_sunspots = pd.read_csv("..\data\monthly-sunspots.csv", header=0, index_col=0, parse_dates=True)
serie_sunspots = dataframe_sunspots.squeeze('columns')
serie_sunspots = serie_sunspots.resample('ME').mean() # Para los meses
serie_sunspots.head()

Month
1749-01-31    58.0
1749-02-28    62.6
1749-03-31    70.0
1749-04-30    55.7
1749-05-31    85.0
Freq: ME, Name: Sunspots, dtype: float64

## Mediana

In [5]:
def imputar_mediana(serie, lista):
    dataframe_mediana = pd.DataFrame({
        'Dataset': [],
        'RMSE': [],
        'MAE': [],
        'MAPE': [],
        'BIAS': []
    })
    for serie_nulo in lista:
        
        # Imputamos con la mediana
        serie_mediana = serie_nulo.fillna(serie_nulo.median())

        # Calculamos la raiz del error cuadrático medio con las series imputadas con la mediana
        rmse_mediana = auxi.RMSE(serie, serie_nulo, serie_mediana)
        # Calculamos el error medio absoluto con las series imputadas con la mediana
        mae_mediana = auxi.MAE(serie, serie_nulo, serie_mediana)
        # Calculamos el error medio absoluto porcentual con las series imputadas con la mediana
        mape_mediana = auxi.MAPE(serie, serie_nulo, serie_mediana)
        # Calculamos el bias con las series imputadas con la mediana
        bias_mediana = auxi.BIAS(serie, serie_nulo, serie_mediana)

        nueva_fila = pd.DataFrame({ 'Dataset': [serie_nulo.name], 'RMSE': [rmse_mediana], 'MAE': [mae_mediana], 'MAPE': [mape_mediana], 'BIAS': [bias_mediana]})
        dataframe_mediana = pd.concat([dataframe_mediana, nueva_fila], ignore_index=True)
    return dataframe_mediana

## Listar

In [6]:
lista_series_temperatura = auxi.leer_series('..\miss_valued_data_temperatura\\')
lista_series_cardiovascular = auxi.leer_series('..\miss_valued_data_cardiovascular\\')
lista_series_sunspots = auxi.leer_series('..\miss_valued_data_manchas_solares\\')

## Escalar datos

### Temperatura

In [7]:
min_max_scaler_temperatura = MinMaxScaler().fit(serie_temperatura.values.reshape(-1,1))
serie_original_temperatura_escalada = auxi.aplicar_escalado(serie_temperatura, min_max_scaler_temperatura)
series_temperatura_escaladas = auxi.escalar(min_max_scaler_temperatura, lista_series_temperatura)

### Cardiovascular

In [8]:
min_max_scaler_cardiovascular = MinMaxScaler().fit(serie_cardiovascular.values.reshape(-1,1))
serie_original_cardiovascular_escalada = auxi.aplicar_escalado(serie_cardiovascular, min_max_scaler_cardiovascular)
series_cardiovascular_escaladas = auxi.escalar(min_max_scaler_cardiovascular, lista_series_cardiovascular)

### Manchas solares

In [9]:
min_max_scaler_sunspots = MinMaxScaler().fit(serie_sunspots.values.reshape(-1,1))
serie_original_sunspots_escalada = auxi.aplicar_escalado(serie_sunspots, min_max_scaler_sunspots)
series_sunspots_escaladas = auxi.escalar(min_max_scaler_sunspots, lista_series_sunspots)

## Imputamos mediana

In [10]:
#dataframe_temperatura_mediana = imputar_mediana(serie_temperatura, lista_series_temperatura)
dataframe_temperatura_mediana = imputar_mediana(serie_original_temperatura_escalada, series_temperatura_escaladas)
#dataframe_cardiovascular_mediana = imputar_mediana(serie_cardiovascular, lista_series_cardiovascular)
dataframe_cardiovascular_mediana = imputar_mediana(serie_original_cardiovascular_escalada, series_cardiovascular_escaladas)
#dataframe_manchas_solares_mediana = imputar_mediana(serie_sunspots, lista_series_sunspots)
dataframe_manchas_solares_mediana = imputar_mediana(serie_original_sunspots_escalada, series_sunspots_escaladas)
dataframe_general_mediana = pd.concat([dataframe_temperatura_mediana, dataframe_cardiovascular_mediana, dataframe_manchas_solares_mediana], ignore_index=True)
dataframe_general_mediana

,Dataset,RMSE,MAE,MAPE,BIAS
0,min_temperature_bateria_10.csv,0.055874,0.014254,3.667503,-0.001819
1,min_temperature_bateria_15.csv,0.063456,0.019671,4.956604,-0.001137
2,min_temperature_bateria_20.csv,0.074127,0.027095,7.074714,-0.006938
3,min_temperature_error_medida_10.csv,0.049684,0.012842,3.155294,0.000281
4,min_temperature_error_medida_15.csv,0.060875,0.019188,4.780325,0.000008
5,min_temperature_error_medida_20.csv,0.068149,0.024552,6.086141,0.001445
6,min_temperature_interferencias_10.csv,0.051471,0.013194,3.202149,0.001325
7,min_temperature_interferencias_15.csv,0.062235,0.019510,4.807837,-0.000255
8,min_temperature_interferencias_20.csv,0.073114,0.026436,6.501589,0.001561
9,min_temperature_mix_10.csv,0.050761,0.012788,3.121034,0.000446
